# 100 cims
Codi per generar el json que es fa servir com a bbdd local a l'app.

In [2]:
import json
import re

import pandas as pd
import requests
from bs4 import BeautifulSoup
import unicodedata
import numpy as np

In [23]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

## Extracció dels cims
Cims descarregats en format JSON de la web de la FEEC. S'uneixen les llistes de cims essencials i no essencials en una de sola.

In [ ]:
def extract_peak_data(file, essencial):
    with open(file, "r") as f:
        data = json.load(f)

    for i, peak in enumerate(data):
        r = requests.get(peak["url"]).text
        soup = BeautifulSoup(r, "html.parser")

        peak["id"] = int(re.search(r"cim_id:\s*([^\n\r]*)", r).group(1).replace("'", ""))
        peak["altitud"] = int(soup.select(f'#post-{peak["id"]} > div.col-12.section-cim-content > div > div > div.col-lg-5.mb-5.order-lg-2.order-1 > div.bg-primari.white.p-4 > div > div:nth-child(4)')[0].text.split()[0])
        peak["comarca"] = soup.select(f'#post-{peak["id"]} > div.col-12.section-cim-content > div > div > div.col-lg-5.mb-5.order-lg-2.order-1 > div.bg-primari.white.p-4 > div > div:nth-child(2)')[0].text.replace("\t", "").replace("\n", "").replace("\r", "")
        peak["utm_x"] = int(soup.select(f'#post-{peak["id"]} > div.col-12.section-cim-content > div > div > div.col-lg-5.mb-5.order-lg-2.order-1 > div.bg-primari.white.p-4 > div > div:nth-child(10)')[0].text)
        peak["utm_y"] = int(soup.select(f'#post-{peak["id"]} > div.col-12.section-cim-content > div > div > div.col-lg-5.mb-5.order-lg-2.order-1 > div.bg-primari.white.p-4 > div > div:nth-child(12)')[0].text)
        peak["essencial"] = essencial

        print(f"Pic {i+1}/{len(data)}: {peak['nombre']}")
    return data

In [ ]:
essencials = extract_peak_data("100cims_feec_essencials.json", essencial=True)

In [ ]:
no_essencials = extract_peak_data("100cims_feec_no_essencials.json", essencial=False)

In [ ]:
cent_cims = essencials + no_essencials
#
# with open("100cims.json", "w") as f:
#     f.write(json.dumps(cent_cims, ensure_ascii=False, indent=4))

## Depuració del JSON de cims de wikipedia
S'associa cada entitat del JSON de wikipedia amb el seu equivalent del JSON de cent_cims. Es fa a partir de les coordenades, considerant-se un match si la diferència és <= 0.00001 graus (~1.11 m). Els 10 cims que no fan match es completen manualment al JSON generat.

In [ ]:
with open("100cims_wikipedia.json") as f:
    cent_cims_w = json.load(f)

len(cent_cims_w["results"]["bindings"])

loc_w = list()
for item in cent_cims_w["results"]["bindings"]:
    lng, lat = item["location"]["value"].replace("Point(", "").replace(")", "").split()
    item["location"]["value"] = {"lat": float(lat), "lng": float(lng)}

In [ ]:
count = 0
for i, cim in enumerate(cent_cims):
    lat, lng = cim["lat"], cim["lng"]
    for cim_w in cent_cims_w["results"]["bindings"]:
        lat_w, lng_w = cim_w["location"]["value"]["lat"], cim_w["location"]["value"]["lng"]
        if abs(lat - lat_w) <= 0.00001 and abs(lng_w - lng) <= 0.00001:
            count += 1
            print(f"Match for {cim['nom']}. {count}/{len(cent_cims)} (evaluated {i + 1}/{len(cent_cims)}).")
            cim["wikidata"] = cim_w["q"]["value"]
            try:
                cim["image"] = cim_w["image"]["value"]
            except KeyError:
                cim["image"] = None

In [ ]:
for cim in cent_cims:
    if not "wikidata" in cim:
        print(cim["nom"])

## Comarca a llista
Es converteix el camp "comarca" a format llista.

In [ ]:
for cim in cent_cims:
    cim["comarca"] = cim["comarca"].split(", ")

## Imatges
S'extreuen les imatges de wikidata per cada cim (si en tenen) i s'afegeixen al JSON. __Les dades d'aquí en endavant han estat preprocessades i reorganitzades a la notebook comarques.ipynb.__

In [3]:
with open("centcims.json", encoding="utf-8") as f:
    cent_cims = json.load(f)
cent_cims

[{'name': 'El Cogulló de Cabra',
  'id': 639,
  'coords': {'latlng': {'lat': 41.4290355033, 'lng': 1.3057579489},
   'utm': {'x': 358437, 'y': 4587771}},
  'topography': {'territory': [{'name': 'Catalunya', 'id': 'CAT'}],
   'vegueria': [{'name': 'Camp', 'id': 3}],
   'comarca': [{'name': 'Alt Camp', 'id': 1}]},
  'essential': True,
  'url': {'feec': 'https://www.feec.cat/activitats/100-cims/cim/el-cogullo-de-cabra/',
   'wikidata': 'http://www.wikidata.org/entity/Q17481698',
   'wikipedia': 'https://ca.wikipedia.org/wiki/El_Cogull%C3%B3_(el_Pont_d%27Armentera)'},
  'images': ['https://upload.wikimedia.org/wikipedia/commons/b/bd/El_Cogull%C3%B3_des_de_Montblanc_P1250216.jpg'],
  'height': 881,
  'summary': "El Cogulló és una muntanya de 881 metres que es troba al municipi del Pont d'Armentera, a la comarca de l'Alt Camp.",
  'ascensions': {'total': 727,
   'monthly': [94, 98, 64, 98, 53, 36, 21, 29, 43, 71, 51, 69]}},
 {'name': 'Puig de Tretzevents',
  'id': 637,
  'coords': {'latlng

In [13]:
from wikidata.client import Client

client = Client()
entity = client.get("Q6650", load=True)
image_prop = client.get("P18")
image = entity[image_prop]
image.title

"File:Vinça, la retenue d'eau et le Canigou.jpg"

In [20]:
metadata_url = "https://en.wikipedia.org/w/api.php?action=query&prop=imageinfo&iiprop=extmetadata&titles={file}&format=json"
r = requests.get(metadata_url.format(file=image.title)).json()
extmetadata = r["query"]["pages"]["-1"]["imageinfo"][0]["extmetadata"]
extmetadata["LicenseUrl"]["value"]
BeautifulSoup(extmetadata["Artist"]["value"]).text
extmetadata["LicenseShortName"]["value"]

'CC BY-SA 3.0'

Els cims sense imatge surten com a `ERROR`, els que tenen tots els paràmetres com a `SUCCESS` i si els hi falta alguna metadada com a `WARNING`. Quan el `WARNING` és `LICENSE URL MISSING` és perquè la llicència és _Public Domain_ i no hi ha cap URL associada perquè no és necessàri.

In [26]:
%%time
count = 0

metadata_url = "https://en.wikipedia.org/w/api.php?action=query&prop=imageinfo&iiprop=extmetadata&titles={file}&format=json"
client = Client()
for i, cim in enumerate(cent_cims):
    error_messages = []

    entity = client.get(cim["url"]["wikidata"].split("/")[-1], load=True)
    try:
        image = entity[client.get("P18")]
        image_url = image.image_url

        try:
            r = requests.get(metadata_url.format(file=image.title)).json()
            extmetadata = r["query"]["pages"]["-1"]["imageinfo"][0]["extmetadata"]
        except KeyError:
            error_messages.append("METADATA MISSING")
            author = None
            license = None
            license_url = None

        try:
            author = BeautifulSoup(extmetadata["Artist"]["value"]).text
        except KeyError:
            error_messages.append("AUTHOR MISSING")
            author = None

        try:
            license = extmetadata["LicenseShortName"]["value"]
        except KeyError:
            error_messages.append("LICENSE MISSING")
            license = None

        try:
            license_url = extmetadata["LicenseUrl"]["value"]
        except KeyError:
            error_messages.append("LICENSE URL MISSING")
            license_url = None

        cim["images"] = [{"image_url": image_url, "author": author, "license": license, "license_url": license_url}]
        count += 1

        if len(error_messages) == 0:
            print(f"{bcolors.OKGREEN}SUCCESS{bcolors.ENDC}: {cim['name']}. {count}/{len(cent_cims)} (evaluated {i + 1}/{len(cent_cims)}).")
        else:
            print(f"{bcolors.WARNING}WARNING{bcolors.ENDC}: {cim['name']}. {count}/{len(cent_cims)} (evaluated {i + 1}/{len(cent_cims)}). {bcolors.WARNING}{', '.join(error_messages)}{bcolors.ENDC}")
    except KeyError:
        print(f"{bcolors.FAIL}ERROR{bcolors.ENDC}: {cim['name']} (evaluated {i + 1}/{len(cent_cims)}).")
        cim["images"] = []

SUCCESS: El Cogulló de Cabra. 1/522 (evaluated 1/522).
SUCCESS: Puig de Tretzevents. 2/522 (evaluated 2/522).
SUCCESS: Torre de Madeloc. 3/522 (evaluated 3/522).
SUCCESS: Cambra d'Ase. 4/522 (evaluated 4/522).
SUCCESS: Carlit. 6/522 (evaluated 6/522).
SUCCESS: Puig Peric. 7/522 (evaluated 7/522).
SUCCESS: Roc de Madres. 8/522 (evaluated 8/522).
SUCCESS: Torre d'Eina. 9/522 (evaluated 9/522).
SUCCESS: Casamanya Nord. 10/522 (evaluated 10/522).
SUCCESS: Pic del Vent. 13/522 (evaluated 13/522).
SUCCESS: Pic Negre d'Envalira. 14/522 (evaluated 14/522).
SUCCESS: Sant Sadurní de Gallifa. 15/522 (evaluated 15/522).
SUCCESS: Tagamanent. 16/522 (evaluated 16/522).
SUCCESS: Tristaina. 17/522 (evaluated 17/522).
SUCCESS: Puigmal. 18/522 (evaluated 18/522).
SUCCESS: Castellsapera. 19/522 (evaluated 19/522).
SUCCESS: El Cogul. 20/522 (evaluated 20/522).
SUCCESS: Les Agudes. 22/522 (evaluated 22/522).
SUCCESS: Maubèrme. 23/522 (evaluated 23/522).
SUCCESS: Montardo. 24/522 (evaluated 24/522).
SUCCESS

## Resum
A partir de l'identificador de wikidata es troba la pàgina de viquipèdia del cim i s'extreu el primer paràgraf. Dels 522 cims, de 50 no ha trobat pàgina de Viquipèdia d'on extreure el resum. D'aquests 50, s'ha trobat manualment que 14 sí que tenien pàgina de Viquipèdia però no devia estar associadaa amb la pàgina de Wikidata, de manera que s'han copiat i enganxat a mà sobre l'arxiu JSON.

In [35]:
with open("centcims2.json", encoding="utf-8") as f:
    cent_cims = json.load(f)
cent_cims

[{'name': 'El Cogulló de Cabra',
  'id': 639,
  'coords': {'latlng': {'lat': 41.4290355033, 'lng': 1.3057579489},
   'utm': {'x': 358437, 'y': 4587771}},
  'topography': {'territory': [{'name': 'Catalunya', 'id': 'CAT'}],
   'vegueria': [{'name': 'Camp', 'id': 3}],
   'comarca': [{'name': 'Alt Camp', 'id': 1}]},
  'essential': True,
  'url': {'feec': 'https://www.feec.cat/activitats/100-cims/cim/el-cogullo-de-cabra/',
   'wikidata': 'http://www.wikidata.org/entity/Q17481698',
   'wikipedia': 'https://ca.wikipedia.org/wiki/El_Cogull%C3%B3_(el_Pont_d%27Armentera)'},
  'images': [{'image_url': 'https://upload.wikimedia.org/wikipedia/commons/b/bd/El_Cogull%C3%B3_des_de_Montblanc_P1250216.jpg',
    'author': 'pere prlpz',
    'license': 'CC BY-SA 3.0',
    'license_url': 'https://creativecommons.org/licenses/by-sa/3.0'}],
  'height': 881,
  'summary': "El Cogulló és una muntanya de 881 metres que es troba al municipi del Pont d'Armentera, a la comarca de l'Alt Camp.",
  'ascensions': {'to

In [ ]:
count = 0

for i, cim in enumerate(cent_cims):
    entity = dict(requests.get(f'https://www.wikidata.org/w/api.php?action=wbgetentities&format=json&props=sitelinks/urls&sitefilter=cawiki&ids={cim["url"]["wikidata"].split("/")[-1]}').json()["entities"]).values()
    sitelinks = list(entity)[0]["sitelinks"]
    if "cawiki" in sitelinks:
        metadata_url = sitelinks["cawiki"]["url"]
        url_name = "%2F".join(metadata_url.split("/")[4:])      # Això ho faig perquè el cim 413 surt a viquipèdia com a "Puig Estela (Ogassa/Pardines)" i la "/" petava el sistma.
        summary = unicodedata.normalize("NFKD", requests.get(f'https://ca.wikipedia.org/api/rest_v1/page/summary/{url_name}').json()["extract"])
        cim["url"]["wikipedia"] = metadata_url
        cim["summary"] = summary

        count += 1
        print(f"Match for {cim['name']}. {count}/{len(cent_cims)} (evaluated {i + 1}/{len(cent_cims)}).")
    else:
        cim["url"]["wikipedia"] = None
        cim["summary"] = None

In [ ]:
json.dumps(cent_cims, ensure_ascii=False)

In [34]:
with open("centcims.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(cent_cims, ensure_ascii=False, indent=4))

## Estacionalitat

In [14]:
df = pd.DataFrame.from_dict(cent_cims)
df.iloc[np.where(df["name"].str.len() == df["name"].str.len().max())]

,name,id,coords,topography,essential,url,images,height,summary,ascensions
0,El Cogulló de Cabra,639,"{'latlng': {'lat': 41.4290355033, 'lng': 1.305...","{'territory': [{'name': 'Catalunya', 'id': 'CA...",True,{'feec': 'https://www.feec.cat/activitats/100-...,[https://upload.wikimedia.org/wikipedia/common...,881,El Cogulló és una muntanya de 881 metres que...,"{'total': 727, 'monthly': [94, 98, 64, 98, 53,..."
1,Puig de Tretzevents,637,"{'latlng': {'lat': 42.492101894, 'lng': 2.4683...","{'territory': [{'name': 'Catalunya Nord', 'id'...",True,{'feec': 'https://www.feec.cat/activitats/100-...,[https://upload.wikimedia.org/wikipedia/common...,2731,El Puig dels Tres Vents o de Tretzevents és u...,"{'total': 198, 'monthly': [1, 1, 2, 5, 4, 34, ..."
2,Torre de Madeloc,634,"{'latlng': {'lat': 42.4904284262, 'lng': 3.075...","{'territory': [{'name': 'Catalunya Nord', 'id'...",True,{'feec': 'https://www.feec.cat/activitats/100-...,[https://upload.wikimedia.org/wikipedia/common...,656,None,"{'total': 513, 'monthly': [34, 26, 34, 56, 56,..."
3,Cambra d'Ase,613,"{'latlng': {'lat': 42.450876, 'lng': 2.1312612...","{'territory': [{'name': 'Catalunya Nord', 'id'...",True,{'feec': 'https://www.feec.cat/activitats/100-...,[https://upload.wikimedia.org/wikipedia/common...,2750,"El Cambra d'Ase, algunes vegades grafiat Cambr...","{'total': 480, 'monthly': [22, 8, 30, 18, 15, ..."
4,Canigó,622,"{'latlng': {'lat': 42.518901928, 'lng': 2.4565...","{'territory': [{'name': 'Catalunya Nord', 'id'...",True,{'feec': 'https://www.feec.cat/activitats/100-...,[https://upload.wikimedia.org/wikipedia/common...,2784,"El Canigó és una muntanya del Pirineu, situa...","{'total': 969, 'monthly': [2, 3, 10, 9, 13, 18..."
...,...,...,...,...,...,...,...,...,...,...
517,Pic de Sallafort,341,"{'latlng': {'lat': 42.4734705922, 'lng': 3.038...","{'territory': [{'name': 'Catalunya', 'id': 'CA...",False,{'feec': 'https://www.feec.cat/activitats/100-...,[],994,"El Pic o Puig de Sallfort, sovint denominat ta...","{'total': 387, 'monthly': [34, 19, 43, 36, 53,..."
518,Puig de la Gavarra,338,"{'latlng': {'lat': 42.3607261639, 'lng': 2.696...","{'territory': [{'name': 'Catalunya', 'id': 'CA...",False,{'feec': 'https://www.feec.cat/activitats/100-...,[],1057,El Puig de la Gavarra és una muntanya de 1057...,"{'total': 366, 'monthly': [29, 22, 17, 38, 47,..."
519,Puigcabrer,335,"{'latlng': {'lat': 41.3153344091, 'lng': 1.191...","{'territory': [{'name': 'Catalunya', 'id': 'CA...",False,{'feec': 'https://www.feec.cat/activitats/100-...,[https://upload.wikimedia.org/wikipedia/common...,524,El Puigcabrer és una muntanya de 524 metres q...,"{'total': 574, 'monthly': [84, 56, 40, 65, 67,..."
520,Montagut d'Ancosa,334,"{'latlng': {'lat': 41.4068776962, 'lng': 1.422...","{'territory': [{'name': 'Catalunya', 'id': 'CA...",False,{'feec': 'https://www.feec.cat/activitats/100-...,[https://upload.wikimedia.org/wikipedia/common...,963,El Montagut és una muntanya de 963 metres que...,"{'total': 698, 'monthly': [73, 65, 63, 72, 143..."


In [17]:
comarques_df = pd.DataFrame(columns=["name", "comarques", "comarques_raw"])
for cim in cent_cims:
    comarques = [comarca["name"] for comarca in cim["topography"]["comarca"]]
    comarques_df.loc[len(comarques_df.index)] = [cim["name"], comarques, "".join(comarques)]
comarques_df

,name,comarques,comarques_raw
0,El Cogulló de Cabra,[Alt Camp],Alt Camp
1,Puig de Tretzevents,"[Conflent, Vallespir]",ConflentVallespir
2,Torre de Madeloc,[Rosselló],Rosselló
3,Cambra d'Ase,[Alta Cerdanya],Alta Cerdanya
4,Canigó,[Conflent],Conflent
...,...,...,...
517,Pic de Sallafort,"[Alt Empordà, Rosselló]",Alt EmpordàRosselló
518,Puig de la Gavarra,[Alt Empordà],Alt Empordà
519,Puigcabrer,[Alt Camp],Alt Camp
520,Montagut d'Ancosa,[Alt Camp],Alt Camp


In [18]:
comarques_df.iloc[np.where(comarques_df["comarques_raw"].str.len() == comarques_df["comarques_raw"].str.len().max())]

,name,comarques,comarques_raw
101,Sant Salvador de les Espases,"[Baix Llobregat, Vallès Occidental]",Baix LlobregatVallès Occidental
468,Puig Madrona,"[Baix Llobregat, Vallès Occidental]",Baix LlobregatVallès Occidental
511,Puig de la Mola,"[Alt Penedès, Baix Llobregat, Garraf]",Alt PenedèsBaix LlobregatGarraf
